In [1]:
import os
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm
from load_djembe_marker import *
from foot_module import onset_calculations, onset_extraction, onset_filtering, utils, onset_plot
# from kde_main import *

In [2]:
OnsetProcessor = onset_calculations.OnsetProcessor()
FootEventDetector = onset_extraction.FootEventDetector()
onset_filtering = onset_filtering.Filtering(mocap_fps = 240)

def auto_calc_time_threshold(data):
    
    D1_diff = np.diff(data/240)
    D1_diff_filtered = D1_diff[(D1_diff >= 0) & (D1_diff <= 1)]
    threshold = np.mean(D1_diff_filtered)
    return threshold

In [4]:
mvnx_filepath = os.listdir("/itf-fi-ml/home/sagardu/djembe_drive/Data/DD_MultiM_Proc/MoCap_proc/MVNX")
dance_annotations = os.listdir("/itf-fi-ml/home/sagardu/djembe_drive/sgr_space/subset_dance_annotation")
mnvx_files = []
cycle_file = []

for filename in dance_annotations:

    filen = filename.split("_Dancers")[0]
    mnvx_files.append(filen+"_T.mvnx")
    cycle_file.append(filen+"_C.csv")

In [5]:
kde_gr = []
kde_in = []
kde_au = []
onset_per_piece_info = []

kde_data_per_piece = {
    "Suku": {"gr": [], "in": [], "au": []},     # all suku recordings
    "Wasulunka": {"gr": [], "in": [], "au": []},
    "Maraka": {"gr": [], "in": [], "au": []},
    "Dansa": {"gr": [], "in": [], "au": []},
    "Sandia": {"gr": [], "in": [], "au": []},
    "Manjanin": {"gr": [], "in": [], "au": []}
}

kde_data_per_dancer = {
    "dancer_1": {"gr": [], "in": [], "au": []},
    "dancer_2": {"gr": [], "in": [], "au": []},
    "dancer_3": {"gr": [], "in": [], "au": []},
    "dancer_4": {"gr": [], "in": [], "au": []},
}

kde_data_all_pieces = {"gr": [], "in": [], "au": []}

for k in tqdm(range(len(dance_annotations))):
    
    file_path = f"/itf-fi-ml/home/sagardu/djembe_drive/Data/DD_MultiM_Proc/MoCap_proc/MVNX/{mnvx_files[k]}"
    file_name = os.path.basename(file_path).split(".")[0]
    piece_name = file_name.split("_")[4]
    ensemble_name = file_name.split("_")[1]
    day = file_name.split("_")[2]
    
    djembe_obj = djembe(file_path)
    print("Loaded:", file_path)
    
    # Load dance annotations
    df_annotation = pd.read_csv(f"/itf-fi-ml/home/sagardu/djembe_drive/sgr_space/subset_dance_annotation/{dance_annotations[k]}")
    print("Loaded:", dance_annotations[k])
    
    # Load meter cycle onsets
    df_metric = pd.read_csv(f"/itf-fi-ml/home/sagardu/djembe_drive/sgr_space/metric_cycles/{cycle_file[k]}")
    loaded_mcycle_onsets = np.round(df_metric["Selected Onset"].to_numpy(), 3)
    print("Loaded:", cycle_file[k])
    print("\n")
    
    motiondata_section, _, _, _ = djembe_obj.load_djembe(frme=-1)                   #   slice(round((start_sec-1)*240), round(end_sec*240))

    # directory creation
    base_logs_folder = "logs"
    piece_folder_name = file_name.split(".")[0]
    piece_folder_path = utils.create_folder(base_logs_folder, piece_folder_name)

    mocap_fps = 240
    foot_segment_name = "Feet"
    ankle_ground_pos = 0.093      # 0.093
    toe_ground_pos = 0.020        # 0.020

    ############# RIGHT ANKLE #################
    Rankle_data = FootEventDetector.foot_detection(motiondata_section, "SEGMENT_RIGHT_FOOT", ankle_ground_pos, foot_ground_threshold = 0.005, time_threshold_seconds= 0.16, zheight_threshold = 0.00)

    Rankle_events_unfiltered = Rankle_data["foot_events_unfiltered"]
    Rankle_events_filtered = Rankle_data["foot_events_filtered"]
    Rshifted_ankle_pos_data = Rankle_data["shifted_foot_pos_data"]

    Rankle_maxima_filtered = Rankle_data["foot_maxima_filtered"]
    Rankle_minima_filtered = Rankle_data["foot_minima_filtered"]

    ############# RIGHT TOE #################
    Rtoe_data = FootEventDetector.foot_detection(motiondata_section, "SEGMENT_RIGHT_TOE", toe_ground_pos, foot_ground_threshold = 0.005, time_threshold_seconds= 0.16, zheight_threshold = 0.00)

    Rtoe_events_unfiltered = Rtoe_data["foot_events_unfiltered"]
    Rtoe_events_filtered = Rtoe_data["foot_events_filtered"]
    Rshifted_toe_pos_data = Rtoe_data["shifted_foot_pos_data"]

    Rtoe_maxima_filtered = Rtoe_data["foot_maxima_filtered"]
    Rtoe_minima_filtered = Rtoe_data["foot_minima_filtered"]

    ############# LEFT ANKLE #################
    Lankle_data = FootEventDetector.foot_detection(motiondata_section, "SEGMENT_LEFT_FOOT", ankle_ground_pos, foot_ground_threshold = 0.005, time_threshold_seconds= 0.16, zheight_threshold = 0.00)

    Lankle_events_unfiltered = Lankle_data["foot_events_unfiltered"]
    Lankle_events_filtered = Lankle_data["foot_events_filtered"]
    Lshifted_ankle_pos_data = Lankle_data["shifted_foot_pos_data"]

    Lankle_maxima_filtered = Lankle_data["foot_maxima_filtered"]
    Lankle_minima_filtered = Lankle_data["foot_minima_filtered"]

    Lankle_vel_data = Lankle_data["foot_vel_data"]

    ############# LEFT TOE #################
    Ltoe_data = FootEventDetector.foot_detection(motiondata_section, "SEGMENT_LEFT_TOE", toe_ground_pos, foot_ground_threshold = 0.005, time_threshold_seconds= 0.16, zheight_threshold = 0.00)

    Ltoe_events_unfiltered = Ltoe_data["foot_events_unfiltered"]
    Ltoe_events_filtered = Ltoe_data["foot_events_filtered"]
    Lshifted_toe_pos_data = Ltoe_data["shifted_foot_pos_data"]

    Ltoe_maxima_filtered = Ltoe_data["foot_maxima_filtered"]
    Ltoe_minima_filtered = Ltoe_data["foot_minima_filtered"]


    ############### ONSET MINIMA FILTERING ##################    
    
    # Thresholds for minima filtering
    pos_diff_threshold_for_minima= 0.02     #   
    max_threshold_for_minima= 0.03          # no movement below this height
    time_diff_threshold_for_minima= 0.2     #   

    Lankle_filtered_minima = onset_filtering.filter_minima(Lankle_maxima_filtered, Lankle_minima_filtered, Lshifted_ankle_pos_data, 
                                                        pos_diff_threshold= pos_diff_threshold_for_minima, 
                                                        max_threshold= max_threshold_for_minima, 
                                                        time_diff_threshold= time_diff_threshold_for_minima)             # minima are non contact end points, looking at ankle is enough

    Rankle_filtered_minima = onset_filtering.filter_minima(Rankle_maxima_filtered, Rankle_minima_filtered, Rshifted_ankle_pos_data, 
                                                        pos_diff_threshold= pos_diff_threshold_for_minima, 
                                                        max_threshold= max_threshold_for_minima, 
                                                        time_diff_threshold= time_diff_threshold_for_minima)

    ######################
    # Create binary representation of ground contact or not, with a height threshold of foot  
    ######################    

    pos_thres_for_ground = 0.025    # Filters out foot movements below this threshold
    
    Rankle_contact_binary, Rankle_event = onset_filtering.create_binary_foot_contact(Rshifted_ankle_pos_data, Rankle_events_filtered, b_value = 1, pos_thres = pos_thres_for_ground)
    Rtoe_contact_binary, Rtoe_event = onset_filtering.create_binary_foot_contact(Rshifted_toe_pos_data, Rtoe_events_filtered, b_value = 1, pos_thres = pos_thres_for_ground)

    Lankle_contact_binary, Lankle_event = onset_filtering.create_binary_foot_contact(Lshifted_ankle_pos_data, Lankle_events_filtered, b_value = 1, pos_thres = pos_thres_for_ground)
    Ltoe_contact_binary, Ltoe_event = onset_filtering.create_binary_foot_contact(Lshifted_toe_pos_data, Ltoe_events_filtered, b_value = 1, pos_thres = pos_thres_for_ground)    

    ######################
    # Detect onsets (where it goes from 1 to 0, meaning no contact to ground contact)
    ######################

    Rankle_onsets = np.where((Rankle_contact_binary[:-1] == 1) & (Rankle_contact_binary[1:] == 0))[0] + 1       # ground contact frames
    Rtoe_onsets = np.where((Rtoe_contact_binary[:-1] == 1) & (Rtoe_contact_binary[1:] == 0))[0] + 1             # ground contact frames

    Lankle_onsets = np.where((Lankle_contact_binary[:-1] == 1) & (Lankle_contact_binary[1:] == 0))[0] + 1       # ground contact frames
    Ltoe_onsets = np.where((Ltoe_contact_binary[:-1] == 1) & (Ltoe_contact_binary[1:] == 0))[0] + 1             # ground contact frames

    # Define foot contact based on ankle and toe onsets with a time threshold: selects first contact (ankle/toe)

    ank_toe_thres_sec = 0.2    # within foot: ankle and toe threshold
    rfoot_onsets, rtoe_heel_diff, rheel_toe_diff = onset_filtering.select_first_foot_contact(Rankle_onsets, Rtoe_onsets, ank_toe_thres_sec = ank_toe_thres_sec)    # right foot Ground onset
    lfoot_onsets, ltoe_heel_diff, lheel_toe_diff = onset_filtering.select_first_foot_contact(Lankle_onsets, Ltoe_onsets, ank_toe_thres_sec = ank_toe_thres_sec)    # left foot Ground onset   

    # Within Foot filtering
    R_foot_minima_cleaned = onset_filtering.filter_minima_between_footstep(rfoot_onsets, Rankle_filtered_minima, threshold_s=0.18)         # Filter out the second onset within 160ms
    L_foot_minima_cleaned = onset_filtering.filter_minima_between_footstep(lfoot_onsets, Lankle_filtered_minima, threshold_s=0.18)         # Filter out the second onset within 160ms

    # Combine both the foot onsets
    both_foot = np.concatenate((R_foot_minima_cleaned, L_foot_minima_cleaned))         # Combine both the foot onsets
    both_foot = np.sort(both_foot)

    both_foot_cleaned = onset_filtering.filter_onsets_by_threshold(both_foot, threshold_s=0.05)       # Filter out the second onset within 50ms across foot
    both_foot_cleaned_t = both_foot_cleaned/240



    ###################################################
    ###################################################
    ################### SAVE INFO #####################
    
    save_onset_params = {
    "base_logs_folder": base_logs_folder,
    "piece_folder_name": piece_folder_name,
    "file_name": file_name,
    "foot_segment_name": foot_segment_name,
    "both_foot_cleaned_t": both_foot_cleaned_t,
    "both_foot_cleaned": both_foot_cleaned,
    "rfoot_onsets": rfoot_onsets,
    "lfoot_onsets": lfoot_onsets,
    "rtoe_heel_diff": rtoe_heel_diff,
    "rheel_toe_diff": rheel_toe_diff,
    "ltoe_heel_diff": ltoe_heel_diff,
    "lheel_toe_diff": lheel_toe_diff,
    "onset_per_piece_info": onset_per_piece_info
    }
    
    onsetinfo_folder_name = "onset_info"
    utils.save_onset_information(save_onset_params, onsetinfo_folder_name)

    
    
    # onsetinfo_folder_name = "onset_info"
    # onsetinfo_folder_path = utils.create_folder(base_logs_folder, piece_folder_name, onsetinfo_folder_name)

    # # Record number of footsteps per piece
    # onset_per_piece_info.append((file_name, len(both_foot_cleaned_t)))

    # #  save onsets as CSV in the mode folder
    # df_both_feet = pd.DataFrame({"time_sec": both_foot_cleaned_t, "frames": both_foot_cleaned})
    # df_rfoot_onsets = pd.DataFrame({"time_sec": rfoot_onsets/240, "frames": rfoot_onsets})
    # df_lfoot_onsets = pd.DataFrame({"time_sec": lfoot_onsets/240, "frames":lfoot_onsets})
    # df_rtoe_heel_timediff = pd.DataFrame({"time_sec": np.concatenate((rtoe_heel_diff/240, rheel_toe_diff/240)), "frames": np.concatenate((rtoe_heel_diff, rheel_toe_diff))})
    # df_ltoe_heel_timediff = pd.DataFrame({"time_sec": np.concatenate((ltoe_heel_diff/240, lheel_toe_diff/240)), "frames": np.concatenate((ltoe_heel_diff, lheel_toe_diff))})
    
    # df_both_feet.to_csv(os.path.join(onsetinfo_folder_path, f"{file_name}_{foot_segment_name}_both_feet_firstcontact_withminima_onsets.csv"), index=False)
    # df_rfoot_onsets.to_csv(os.path.join(onsetinfo_folder_path, f"{file_name}_{foot_segment_name}_right_foot_firstcontact_nominima_onsets.csv"), index=False)
    # df_lfoot_onsets.to_csv(os.path.join(onsetinfo_folder_path, f"{file_name}_{foot_segment_name}_left_foot_firstcontact_nominima_onsets.csv"), index=False)
    # df_rtoe_heel_timediff.to_csv(os.path.join(onsetinfo_folder_path, f"{file_name}_{foot_segment_name}_right_heel_toe_timediff.csv"), index=False)
    # df_ltoe_heel_timediff.to_csv(os.path.join(onsetinfo_folder_path, f"{file_name}_{foot_segment_name}_left_heel_toe_timediff.csv"), index=False)
    
    ###################################################
    ###################################################
    ################### SAVE PLOTS #####################

    # directory creation
    plot_folder_name = "saved_plots_for_inspection"
    plot_folder_path = utils.create_folder(base_logs_folder, piece_folder_name, plot_folder_name)

    # onset_plot.py Inspect_plots is class
    inspect_plots = onset_plot.Inspect_plots(file_name, mocap_fps, plot_folder_path, max_threshold_for_minima,
                    pos_diff_threshold_for_minima, pos_thres_for_ground, time_diff_threshold_for_minima, 
                    ank_toe_thres_sec)       # Initialize with parameters

    inspect_plots.inspect_filtered_minima(Rshifted_ankle_pos_data, Rankle_maxima_filtered, Rankle_filtered_minima, 
                                        Lshifted_ankle_pos_data, Lankle_maxima_filtered, Lankle_filtered_minima)

    inspect_plots.inspect_ground_contact_onsets(Rankle_contact_binary, Rtoe_contact_binary, rfoot_onsets, 
                                        Lankle_contact_binary, Ltoe_contact_binary, lfoot_onsets)

    inspect_plots.inspect_first_ground_contact_and_minima_onsets(Rankle_contact_binary, rfoot_onsets, Rankle_filtered_minima, lfoot_onsets, Lankle_filtered_minima)

    inspect_plots.inspect_first_ground_contact_and_filtered_minima(Rankle_contact_binary, 
                                                    rfoot_onsets, lfoot_onsets, 
                                                    R_foot_minima_cleaned, L_foot_minima_cleaned)

    inspect_plots.inspect_both_foot(Rankle_contact_binary, both_foot, both_foot_cleaned)

    #########################################################
    #########################################################
    ############ Kernel Density Estimation Part #############
    
    kde_folder_name = "average_kde_plots"
    kde_folder_path = utils.create_folder(base_logs_folder, piece_folder_name, kde_folder_name)
    
    
    modes = df_annotation["mocap"].unique().tolist()
    
    
    kde_list = []   # collect feet_onsets for the 3 modes for subplot
    for mocap_mode in modes:

        category_df = df_annotation.groupby('mocap')
        category_df = category_df.get_group(mocap_mode)
        category_df = category_df.reset_index(drop=True)

        choose_nb_onset_to_make_block = 2
        section_data = OnsetProcessor.onset_calculations(category_df, loaded_mcycle_onsets, choose_nb_onset_to_make_block)

        # directoru creation
        mode_folder_path = utils.create_folder(base_logs_folder, piece_folder_name, mocap_mode)
        
        utils_func = utils.Utils_func(section_data, file_name, foot_segment_name, mocap_mode, mode_folder_path, kde_folder_path)
        feet_onsets = utils_func.collect_foot_onsets_per_cycle(both_foot_cleaned_t)
        
        kde = utils_func.kde_plot_save(feet_onsets)     # average kde
        kde_list.append(kde)    # for subplotting the 3 modes
        
        kde_data_all_pieces[mocap_mode].append(kde)     # kde across all pieces
        kde_data_per_piece[piece_name][mocap_mode].append(kde)  # kde per piece type

        if (ensemble_name == "E1" and day == "D1") or (ensemble_name == "E1" and day == "D5"):
            kde_data_per_dancer["dancer_1"][mocap_mode].append(kde)
        if (ensemble_name == "E1" and day == "D2"):
            kde_data_per_dancer["dancer_2"][mocap_mode].append(kde)
        if (ensemble_name == "E2" and day == "D3") or (ensemble_name == "E2" and day == "D4"):
            kde_data_per_dancer["dancer_3"][mocap_mode].append(kde)
        if (ensemble_name == "E3" and day == "D5") or (ensemble_name == "E3" and day == "D6"):
            kde_data_per_dancer["dancer_4"][mocap_mode].append(kde)


    utils_func.kde_subplot_save(kde_list, modes, kde_folder_path)    # for subplotting the 3 modes kde for each piece
    utils_func.generate_foot_audio_click(Rankle_contact_binary, both_foot_cleaned_t, pos_thres_for_ground, piece_folder_path)

    if k ==5:
        break

# save kde averaged across different combinations
kde_piece_dancer_path = utils.create_folder(base_logs_folder, "kde_plots_per_piece_and_dancer")
modes_list = ["gr", "in", "au"]

# Create and save KDE plots for each piece
for piece_name, kde_data in kde_data_per_piece.items():
    utils_func.kde_per_piece_dancer_subplot_save(kde_data, piece_name, modes_list, kde_piece_dancer_path)

# Create and save KDE plots for each dancer
for dancer_name, kde_data in kde_data_per_dancer.items():
    utils_func.kde_per_piece_dancer_subplot_save(kde_data, dancer_name, modes_list, kde_piece_dancer_path)

# compute and plot average KDE for all pieces
utils_func.saveplot_all_pieces_kde(kde_data_all_pieces, kde_piece_dancer_path)

# # save
# df_onset_info = pd.DataFrame(onset_per_piece_info, columns=['Filename', 'Number of Onsets'])
# df_onset_info.to_csv(os.path.join(onsetinfo_folder_path,"Onset_info_per_piece.csv"), index=False)

  0%|          | 0/47 [00:00<?, ?it/s]

Loaded: /itf-fi-ml/home/sagardu/djembe_drive/Data/DD_MultiM_Proc/MoCap_proc/MVNX/BKO_E1_D1_08_Suku_T.mvnx
Loaded: BKO_E1_D1_08_Suku_Dancers.csv
Loaded: BKO_E1_D1_08_Suku_C.csv


Folder 'logs/BKO_E1_D1_08_Suku_T' created successfully.
Folder 'logs/BKO_E1_D1_08_Suku_T/onset_info' created successfully.
Folder 'logs/BKO_E1_D1_08_Suku_T/saved_plots_for_inspection' created successfully.
Folder 'logs/BKO_E1_D1_08_Suku_T/average_kde_plots' created successfully.
Folder 'logs/BKO_E1_D1_08_Suku_T/in' created successfully.
LinAlgError encountered for block 51. Skipping KDE for this block.


  2%|▏         | 1/47 [01:44<1:20:26, 104.92s/it]

Loaded: /itf-fi-ml/home/sagardu/djembe_drive/Data/DD_MultiM_Proc/MoCap_proc/MVNX/BKO_E3_D5_03_Wasulunka_T.mvnx
Loaded: BKO_E3_D5_03_Wasulunka_Dancers.csv
Loaded: BKO_E3_D5_03_Wasulunka_C.csv


Folder 'logs/BKO_E3_D5_03_Wasulunka_T' created successfully.
Folder 'logs/BKO_E3_D5_03_Wasulunka_T/onset_info' created successfully.
Folder 'logs/BKO_E3_D5_03_Wasulunka_T/saved_plots_for_inspection' created successfully.
Folder 'logs/BKO_E3_D5_03_Wasulunka_T/average_kde_plots' created successfully.
Folder 'logs/BKO_E3_D5_03_Wasulunka_T/gr' created successfully.
Folder 'logs/BKO_E3_D5_03_Wasulunka_T/au' created successfully.
Folder 'logs/BKO_E3_D5_03_Wasulunka_T/in' created successfully.


  4%|▍         | 2/47 [04:21<1:41:36, 135.47s/it]

...

Loaded: /itf-fi-ml/home/sagardu/djembe_drive/Data/DD_MultiM_Proc/MoCap_proc/MVNX/BKO_E2_D4_03_Wasulunka_T.mvnx
Loaded: BKO_E2_D4_03_Wasulunka_Dancers.csv
Loaded: BKO_E2_D4_03_Wasulunka_C.csv


Folder 'logs/BKO_E2_D4_03_Wasulunka_T' created successfully.
Folder 'logs/BKO_E2_D4_03_Wasulunka_T/onset_info' created successfully.
Folder 'logs/BKO_E2_D4_03_Wasulunka_T/saved_plots_for_inspection' created successfully.
Folder 'logs/BKO_E2_D4_03_Wasulunka_T/average_kde_plots' created successfully.
Folder 'logs/BKO_E2_D4_03_Wasulunka_T/gr' created successfully.
Folder 'logs/BKO_E2_D4_03_Wasulunka_T/au' created successfully.
Folder 'logs/BKO_E2_D4_03_Wasulunka_T/in' created successfully.
LinAlgError encountered for block 13. Skipping KDE for this block.


 11%|█         | 5/47 [10:26<1:26:19, 123.32s/it]

Loaded: /itf-fi-ml/home/sagardu/djembe_drive/Data/DD_MultiM_Proc/MoCap_proc/MVNX/BKO_E1_D2_04_Maraka_T.mvnx
Loaded: BKO_E1_D2_04_Maraka_Dancers.csv
Loaded: BKO_E1_D2_04_Maraka_C.csv


Folder 'logs/BKO_E1_D2_04_Maraka_T' created successfully.
Folder 'logs/BKO_E1_D2_04_Maraka_T/onset_info' created successfully.
Folder 'logs/BKO_E1_D2_04_Maraka_T/saved_plots_for_inspection' created successfully.
Folder 'logs/BKO_E1_D2_04_Maraka_T/average_kde_plots' created successfully.
Folder 'logs/BKO_E1_D2_04_Maraka_T/gr' created successfully.
Folder 'logs/BKO_E1_D2_04_Maraka_T/au' created successfully.
Folder 'logs/BKO_E1_D2_04_Maraka_T/in' created successfully.


 11%|█         | 5/47 [12:08<1:42:02, 145.78s/it]


Folder 'logs/kde_plots_per_piece_and_dancer' created successfully.
Separate plot saved at logs/kde_plots_per_piece_and_dancer
Separate plot saved at logs/kde_plots_per_piece_and_dancer
Separate plot saved at logs/kde_plots_per_piece_and_dancer
Combined subplot saved at logs/kde_plots_per_piece_and_dancer/All_Pieces_Average_KDE_Subplots.png


In [8]:
motiondata_section['orientation'].keys()



dict_keys(['SEGMENT_PELVIS', 'SEGMENT_L5', 'SEGMENT_L3', 'SEGMENT_T12', 'SEGMENT_T8', 'SEGMENT_NECK', 'SEGMENT_HEAD', 'SEGMENT_RIGHT_SHOULDER', 'SEGMENT_RIGHT_UPPER_ARM', 'SEGMENT_RIGHT_FOREARM', 'SEGMENT_RIGHT_HAND', 'SEGMENT_LEFT_SHOULDER', 'SEGMENT_LEFT_UPPER_ARM', 'SEGMENT_LEFT_FOREARM', 'SEGMENT_LEFT_HAND', 'SEGMENT_RIGHT_UPPER_LEG', 'SEGMENT_RIGHT_LOWER_LEG', 'SEGMENT_RIGHT_FOOT', 'SEGMENT_RIGHT_TOE', 'SEGMENT_LEFT_UPPER_LEG', 'SEGMENT_LEFT_LOWER_LEG', 'SEGMENT_LEFT_FOOT', 'SEGMENT_LEFT_TOE'])